<a href="https://colab.research.google.com/github/Yuri-Nassar/mat_tree/blob/master/TCC_multiAspectClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install libs

In [1]:
!pip install plotly==5.10 -q

## Import libs

In [2]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import math
from copy import deepcopy
from collections import namedtuple, defaultdict
import itertools

from graphviz import Digraph
# import plotly.io as pio
import plotly.graph_objects as go
from IPython.display import display#, display_html, HTML
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()
sns.set(rc={'figure.figsize':(10.7,7.27)})

In [3]:
# dataset path
try:
  from google.colab import drive
  drive.mount('/content/drive')
  try:
    %cd drive/MyDrive/Colab\ Notebooks/mattree-demo
  except:
    pass

  # dataset_path = "drive/MyDrive/Colab Notebooks/mattree-demo/datasets"
  dataset_path = "datasets"
except:
  # locally
  dataset_path = "datasets"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/mattree-demo


# Load dataset

In [4]:
# Number of users in the dataset (#10 or #193)
fs_version = 'fsny_10'

df = pd.read_csv(dataset_path+"/"+fs_version+"/foursquare.csv")
df.head()

,tid,time,day,poi,type,root_type,rating,weather,label
0,5184,1056,Tuesday,my bed,Strip Club,Nightlife Spot,-1.0,Clear,185
1,5184,1099,Tuesday,nanas :),American Restaurant,Food,-1.0,Clear,185
2,5184,1191,Tuesday,The Bush Hot Tub :),Pool,Outdoors & Recreation,-1.0,Clear,185
3,5184,1261,Tuesday,The Hang Out,Other Nightlife,Nightlife Spot,-1.0,Clouds,185
4,5184,1320,Tuesday,The Bush Hideout,Speakeasy,Nightlife Spot,-1.0,Clouds,185


In [5]:
# # matca_demo = TreeNode(input=df)
# # matca_demo.dashboard(TreeNode.df_dict)

# TreeNode.dashboard(df)
# # TreeNode.dashboard(TreeNode.df_dict)

In [6]:
df.label.nunique()

10

# Algorithm

### Sankey

In [7]:
def sankey(cls, label, value):
  """
    Creates a sankey diagram from class dataset. Sankey diagram is a type of
    flow diagram in which the width of the arrows is proportional to the
    flow rate.
  """
  # print(f'label: {label}')
  # print(f'value: {value}')
  # print(f'source: {cls.source}')
  # print(f'id_dict: {cls.id_dict}')
  # print(f'nodeLabel: {cls.nodeLabel}')


  ### give an unique id for labels since it can repeat
  label_to_id = {}
  id = 0
  for k, v in label.items():
    for i in range(len(v)):
      label_to_id[v[i]] = id
      id+=1
  # print(f'label_to_id: {label_to_id}')

  ## source-target-count
  source = []
  target = []
  count = []
  for k, v in label.items():
    id_next_node = 0
    for label_name in v:
      counter = 0
      for i in cls.nodeLabel[k+1]:
        if (counter <= 1) & (label_name.split('#')[0] != 'c'):
          source.append(label_name)
          target.append(label[k+1][id_next_node])
          count.append(value[k][id_next_node])
          id_next_node+=1
          counter+=1
        else:
          break

  data_link = {'source':source,
              'target':target,
              'count':count}
  # print(f'data_link: {data_link}')

  df_source_target = pd.DataFrame(data_link, columns=['source','target','count'])
  # display(df_source_target)
  df_source_target = df_source_target.replace({'source':label_to_id, 'target':label_to_id})
  # display(df_source_target)


  label = list(itertools.chain.from_iterable(label.values()))
  value = list(itertools.chain.from_iterable(value.values()))
  source = [val for val in cls.source for _ in (0, 1)]
  target = [tar for tar in range(1, len(source) + 1)]

  # link = dict(source=source, target=target, value=value)
  # node = dict(label=label, pad=20, thickness=5)
  link = dict(source=df_source_target['source'].values, target=df_source_target['target'],
              value=df_source_target['count'])
  node = dict(label=list(label_to_id.keys()), pad=20, thickness=5)

  data = go.Sankey(link=link, node=node
                   ,arrangement='snap'
                  #  ,orientation='h'
                   ,valueformat = ".0f",)
  fig = go.FigureWidget(data=data)
  # fig.update_layout(title_text="Basic Sankey Diagram", font_size=10,plot_bgcolor='black',
  #                   paper_bgcolor='black',
  #                   width=500, height=500)
  fig.update_layout(width=1000, height=600)
  fig.show()

### EDA

In [8]:
def eda(cls, dataset, feature, usr=None):
  """
  Exploratory Data Analysis. It generates a plot bar of a given feature of
  a given dataset and a given user.

  Parameters
  ----------
  dataset : pandas.DataFrame
    Dataset of trajectories of a given cluster.
  feature : str
    Feature or Aspect intended for EDA.
  user : int
    User label in a given cluster.
  """
  idx = dataset.index

  df_total = df.copy()
  df1 = df[df.tid.isin(idx)]

  if usr is not None and usr is not 'ALL':
    df_total = df_total[df_total.label == usr]
    df1 = df1[df1.label == usr]


  df_total.reset_index(inplace=True)
  df1.reset_index(inplace=True)

  place_type_abs = df_total[feature]
  place_type = df1[feature]


  ''' Frequency per cluster '''
  values = np.sort(np.unique(place_type_abs))
  freq = np.zeros(len(values))
  ind = 0
  for i in values:
      counter = 0
      for j in range(0,len(place_type)):
          if(place_type[j] == i):
            counter = counter + 1
      freq[ind] = counter
      ind = ind + 1

  ''' Absolute frequency '''
  values_abs = np.sort(np.unique(place_type_abs))
  freq_abs = np.zeros(len(values_abs))
  ind_abs = 0
  for k in values_abs:
      counter = 0
      for l in range(0,len(place_type_abs)):
          if(place_type_abs[l] == k):
            counter = counter + 1
      freq_abs[ind_abs] = counter
      ind_abs = ind_abs + 1

  relative = []
  for i in range(len(freq)):
    relative.append(f'{(freq[i]/freq_abs[i])*100}')

  fig= plt.figure(figsize=(15,5))
  ax = plt.gca()


  y_pos = np.arange(len(values))
  plt.xticks(y_pos, values)
  plt.bar(y_pos, freq)
  plt.xticks(fontsize=10, rotation='vertical')
  plt.yticks(fontsize=10)
  plt.xlabel("Aspect values",fontsize = 15)
  plt.ylabel("Frequency", fontsize = 15)
  plt.title(f"Relative frequency for aspect '{feature}'")

  for i, p in enumerate(ax.patches):
      ax.text(p.get_x() + p.get_width()/2., p.get_height(), f'{int(p.get_height())} ({relative[i]:.4}%)',
              fontsize=10, color='black', ha='center', va='bottom')

### EDA CORR

In [9]:
def eda_corr(cls, dataset, usr):
  """
  Exploratory Data Analysis. It generates a plot of correlation matrix of
  all features of a given dataset and a given user.

  Parameters
  ----------
  dataset : pandas.DataFrame
    Dataset of trajectories of a given cluster.
  user : int
    User label in a given cluster.
  """
  idx = dataset.index
  ds = df[df.tid.isin(idx)]

  if usr is not None:
    ds = ds[ds.label == usr]

  columns = ['tid', 'time', 'poi', 'type', 'rating', 'label', 'weather']
  ds = ds.drop(columns=columns)
  corr = pd.get_dummies(ds)
  corr = corr.corr()

  plt.figure(figsize=(10, 4))
  plt.imshow(corr, cmap='Blues', interpolation='none', aspect='auto')
  plt.colorbar()
  plt.xticks(range(len(corr)), corr.columns, rotation='vertical')
  plt.yticks(range(len(corr)), corr.columns);
  plt.suptitle('Correlation between variables', fontsize=15, fontweight='bold')
  plt.grid(False)

### Entropy

In [10]:
def get_entropy(dataset):
  """
    Calculates the entropy value of a given dataset.

    Parameters
    ----------
    dataset : pandas.DataFrame
      Dataset of trajectories of a given cluster

    Returns
    -------
    Float
      Entropy value of a given cluster.
  """
  df1 = dataset.copy()
  entropy_dict = {}
  for e in df1.label.unique():
    num_traj = df1[df1.label == e].tid.unique()
    entropy_dict[e] = len(num_traj)
  total = sum(entropy_dict.values())

  entropy_value = 0
  for key, value in entropy_dict.items():
    p = value/total
    entropy_value += p * math.log2(p)

  try:
    max_entropy = math.log2(len(df1.label.unique()))
    return -entropy_value/max_entropy
  except:
    return -entropy_value
  # return -entropy_value/max_entropy

### Similarity Matrix

In [11]:
def get_similarity_matrix(dataset, sim_measure):
    """
      Creates the distance matrix of the trajectories of a given cluster using
      the given similarity metric.

      Parameters
      ----------
      dataset : pandas.DataFrame
        Dataset of trajectories of a given cluster
      sim_measure : str
        Similarity metric [MUITAS, MSM, EDR, LCSS].

      Returns
      -------
      pandas.DataFrame
        A dataframe of [MUITAS, MSM, EDR, LCSS] similarity metric.
    """

    df_tid = dataset.tid.unique()
    cols = []
    # Select columns according to trajectories
    for e in df_tid:
        cols.append(str(e))

    df_aux = None
    sim_matrix = None
    if sim_measure == 'MUITAS':
        sm_file = dataset_path+"/"+fs_version+"/muitas_"+fs_version.split('_')[-1]+".csv"
        sim_matrix = pd.read_csv(sm_file)#, usecols=cols)
        # df_aux = pd.read_csv(sm_file, usecols=cols).copy()
        # sim_matrix = df_muitas[cols]
        # df_aux = df_muitas.copy()
    elif sim_measure == 'MSM':
        sm_file = dataset_path+"/"+fs_version+"/msm_"+fs_version.split('_')[-1]+".csv"
        sim_matrix = pd.read_csv(sm_file)#, usecols=cols)
        # df_aux = pd.read_csv(sm_file, usecols=cols).copy()
    elif sim_measure == 'EDR':
        sm_file = dataset_path+"/"+fs_version+"/edr_"+fs_version.split('_')[-1]+".csv"
        sim_matrix = pd.read_csv(sm_file)#, usecols=cols)
        # df_aux = pd.read_csv(sm_file, usecols=cols).copy()
    elif sim_measure == 'LCSS':
        sm_file = dataset_path+"/"+fs_version+"/lcss_"+fs_version.split('_')[-1]+".csv"
        sim_matrix = pd.read_csv(sm_file)#, usecols=cols)
        # df_aux = pd.read_csv(sm_file, usecols=cols).copy()

    sim_matrix.index = sim_matrix.columns
    sim_matrix = sim_matrix[cols]
    sim_matrix = sim_matrix.loc[cols]

    # idx = []
    # # Select rows according to trajectories
    # for c in sim_matrix:
    #     idx.append(df_aux.columns.get_loc(c))
    # sim_matrix = sim_matrix[sim_matrix.index.isin(idx)]

    del df_aux, df_tid, cols#,  idx

    return sim_matrix.set_index(sim_matrix.columns)

### TreeNode

In [12]:
class TreeNode:
    """
      A class used to represent a node in a Tree.

      Attributes
      ----------
      TODO

      Methods
      -------
      set_level: Defines the cluster label.
      check_label: Method used to verify cluster aspect label in order to avoid
                   duplicate names in Sankey Diagram.
      sankey: Creates a sankey diagram from class dataset. Sankey diagram is a
              type of flow diagram in which the width of the arrows is
              proportional to the flow rate.
      eda: Exploratory Data Analysis. It generates a plot bar of a given feature of
            a given dataset and a given user.
      eda_corr: Exploratory Data Analysis. It generates a plot of correlation
                matrix of all features of a given dataset and a given user.
      get_similarity_matrix: Creates the distance matrix of the trajectories of
                             a given cluster using the given similarity metric.
      get_entropy: Calculates the entropy value of a given dataset.
      dashboard: Displays the result dashboard.
      show: Shows info about each cluster node.
      graphicTree: Shows info about each cluster node in the tree generated by Digraph
                  plot.
      dashTree: Method that calculates the frequency matrix and the clusters
                generated from the division of data based on a split criteria
                defined in the Tree constructor.

    """

    # Select option in dashboard
    SELECT = 'Select Node'
    ALL = 'ALL'
    HEATMAP = 'HEATMAP'

    # Sankey diagram params
    label = defaultdict(list)
    value = defaultdict(list)
    source = []

    id_dict = defaultdict(list)
    id_list = []

    # Params to label tree nodes
    nodeNum, targetNum = -1, -1
    nodeLabel = defaultdict(list)

    # 1. Dictionary of all nodes dataframes
    # 2. Dictionary of all leaves nodes dataframes
    df_dict = {}
    df_leaves = {}

    ############################################################################
    # Frequency matrix for the initial dataset
    absolute_frequency_matrix = None
    relative = True

    ############################################################################
    temporario = 0
    clusters = 0
    dendrogram_dict = defaultdict(list)

    ############################################################################
    var_dict = {}
    id_iter = itertools.count()

    temp = {}
    # d3 = D3Blocks()

    def __init__(self, df, par=None):
        """DOC - __init__"""

        self.parent = par
        self.parentName = ''
        self.data = df
        self.left = None
        self.leftChildName = ''
        self.right = None
        self.rightChildName = ''
        self.done = 'No'
        self.freqMatrix = None
        self.variance = None
        self.threshold = None
        self.left_group = {}
        self.right_group = {}

        self.division = ''
        self.thresholdVal = None
        self.maxTrajPerGroup = 50
        self.maxDepth = 2

        if par == None:
            # binary, minVariance, var_red, max_red
            self.split = 'max_red'
        else:
            self.split = par.split

        self.trajList = self.data.tid.unique()
        self.useCol = ['day', 'weather', 'root_type']

        if self.parent is not None:
            self.skipVal = deepcopy(self.parent.skipVal)
        else:
            self.skipVal = []

        if self.parent is None:
            self.depth = 0
        else:
            self.depth = par.depth + 1

        if par is None:
            self.id = f'Lvl {self.depth}'
        else:
            self.id = self.set_level(self.depth)

    def set_level(self, depth):
        """
        Defines the cluster label.

        Parameters
        ----------
        depth : int
          Tree depth level.
        """
        id = 0
        while True:
            if id not in self.id_dict[depth]:
                self.id_dict[depth].append(id)
                return f'Lvl {depth} - {id}'
            if id in self.id_dict[depth]:
                id += 1

    def check_label(self, label, depth):
        """
        Method used to verify cluster aspect label in order to avoid duplicate
        names in Sankey Diagram.

        Parameters
        ----------
        label : str
          Cluster aspect label.
        depth : int
          Cluster depth level.
        """

        items = list(itertools.chain.from_iterable(dict(TreeNode.label).values()))

        if label in items:
            if '#' in label:
                label, num = label.split('#')
                label = label+"#"+str(int(num)+1)
                self.check_label(label, depth)
            else:
                label += f'#1'
                self.check_label(label, depth)
        else:
            TreeNode.label[depth].append(label)



    def generate_freq_matrix(self):

        columns = self.data.drop(['time', 'poi', 'type', 'rating', 'label', 'weather'], axis=1)
        dummies = pd.get_dummies(columns, prefix_sep='~')
        vals = dummies.drop(['tid'], axis=1)
        self.freqMatrix = pd.pivot_table(dummies, index=['tid'], values=vals.columns, aggfunc=np.sum)

    @classmethod
    def dashboard(cls, array, strat_split, min_trajs, max_depth):
        """
          Displays the result dashboard.

          output: displays the frequency matrix of a given cluster
          output_3: displays the graphic tree
          output_4: displays the Sankey Diagram
          output_5: displays the cluster pandas.Dataset and info about the cluster
          output_6: displays EDA plots of a given cluster
          output_8: displays the similarity matrix of a given cluster
          plot_output: displays the heat map plots of a given cluster

          Parameters
          ----------
          array : dict
            Dictionary of resulting nodes.

        """
        root = cls(df)
        root.dashTree(strat_split, min_trajs, max_depth)

        def getDictKeys(array_dict):
            keys = list(array_dict.keys())
            keys.sort()
            keys.insert(0, cls.SELECT)
            keys.insert(1, cls.ALL)
            return keys

        def getUsers(data_frame):
            user = data_frame.label.unique().tolist()
            user.insert(0, cls.ALL)
            return user

        dropdown_df = widgets.Dropdown(options=getDictKeys(TreeNode.df_dict), description='Tree Node')
        row_num = widgets.IntText(value=5, step=1, description='# rows')
        dropdown_label = widgets.Dropdown(options=getUsers(df), description='Select User')
        item_layout = widgets.Layout(margin='0 0 10px 0')

        input_widgets = widgets.HBox([dropdown_df, row_num, dropdown_label], layout=item_layout)

        output = widgets.Output()
        # output_2 = widgets.Output()
        output_3 = widgets.Output()
        output_4 = widgets.Output()
        output_5 = widgets.Output()
        output_6 = widgets.Output()
        # output_7 = widgets.Output()
        output_8 = widgets.Output()
        plot_output = widgets.Output()

        tab = widgets.Tab([output_3, output_5, output_8, output_6, output, output_4, plot_output], layout=item_layout)

        def update_dashboard(dataset, num, user):

            output.clear_output()
            # output_2.clear_output()
            output_3.clear_output()
            output_4.clear_output()
            output_5.clear_output()
            output_6.clear_output()
            # output_7.clear_output()
            output_8.clear_output()
            plot_output.clear_output()

            with output:
                if (dataset == cls.ALL) or (dataset == cls.SELECT):
                    display(df.head(num))
                else:
                    display(TreeNode.df_dict[dataset].head(num))

            # graph
            with output_3:
                graph = Digraph(node_attr={'fixedsize':'true', 'width':'1.5',
                                           'height':'.8', 'fontsize':'8','fontname':'bold'})
                # graph.attr(size="40")
                graph.attr()
                root.graphicTree(graph)

                display(graph)

            with output_4:
                sankey(cls, TreeNode.label, TreeNode.value)
                # plt.show()

            with output_5:
                if (dataset == cls.ALL) or (dataset == cls.SELECT):
                    display(df.head(num))
                else:
                    idx = TreeNode.df_dict[dataset].index
                    df_filter = df[df.tid.isin(idx)]
                    print(f'The users in this node are: {df_filter.label.unique()}')
                    print(f'# users: {len(df_filter.label.unique())}')
                    print(f'There are {len(df_filter.tid.unique())} trajectories in this node.\n')

                    print(f'Entropy (Trajectories/User): {get_entropy(df_filter)}\n')

                    if user == 'ALL':
                        display(df_filter.head(num))

                    if user in df_filter.label.unique():
                        dt = df_filter[df_filter.label == user]
                        total = len(df[df.label==user].tid.unique())
                        local = len(dt.tid.unique())
                        print(
                            f"""Showing results for user {user}. User {user} has {local} trajectories in this node of  {total} ({np.round((local/total)*100,2)}%)."""
                            )
                        print(f'Trajectory(ies) are: {[x for x in dt.tid.unique()]}')
                        display(dt.head(num))
                    else:
                        if user != 'ALL':
                            print('This user is not present in this node.')

            with output_6:
                # if user == 'ALL':
                #   dataset = 'Lvl 0'

                if user == 'ALL':
                    eda(cls, TreeNode.df_dict['Lvl 0'], 'root_type', usr=None)
                else:
                    eda(cls, TreeNode.df_dict[dataset], 'root_type', usr=user)

                if user == 'ALL':
                    eda(cls, TreeNode.df_dict['Lvl 0'], 'day', usr=None)
                else:
                    eda(cls, TreeNode.df_dict[dataset], 'day', usr=user)

                if user == 'ALL':
                    eda_corr(cls, TreeNode.df_dict['Lvl 0'], usr=None)
                else:
                    eda_corr(cls, TreeNode.df_dict[dataset], usr=user)
                plt.show()

            # with output_7:
            #     display(TreeNode.var_dict[dataset])

            with output_8:
                if (dataset == cls.ALL) or (dataset == cls.SELECT):
                    print("Select node.")
                else:
                    idx = TreeNode.df_dict[dataset].index
                    df_filter = df[df.tid.isin(idx)]
                    print(f"Users in this node:{df_filter.label.unique()}")

                    if user != 'ALL':
                      df_user = df_filter[df_filter.label == user]
                      muitas_sm = get_similarity_matrix(df_user, 'MUITAS')
                    else:
                      muitas_sm = get_similarity_matrix(df_filter, 'MUITAS')

                    # trajs_id = df_filter.tid.astype(str).unique().tolist()

                    similarity_mean_node_muitas = get_similarity_matrix(df_filter, 'MUITAS')
                    similarity_mean_node_muitas = sum(similarity_mean_node_muitas.mean()) / len(
                        similarity_mean_node_muitas)

                    similarity_mean_node_msm = get_similarity_matrix(df_filter, 'MSM')
                    similarity_mean_node_msm = sum(similarity_mean_node_msm.mean()) / len(similarity_mean_node_msm)

                    print(f'MUITAS')
                    print(
                        f'The average of similarity between the trajectories of this cluster/node is: {similarity_mean_node_muitas}')
                    print(f'MSM')
                    print(
                        f'The average of similarity between the trajectories of this cluster/node is: {similarity_mean_node_msm}')

                    muitas_user_avg = sum(muitas_sm.mean())/len(muitas_sm)
                    print(f'\nSimilarity Matrix (user:{user}; MUITAS: {muitas_user_avg}):')

                    # display(get_similarity_matrix(df_filter, 'MUITAS').head(num))
                    display(muitas_sm.head(num))

            ### heatmap ###
            with plot_output:
                plt.figure(figsize=(10,4))
                if dataset == cls.ALL:
                    ax = sns.heatmap(df)
                    ax.set_yticklabels(df.tid, size = 12)
                else:
                    ax = sns.heatmap(TreeNode.df_dict[dataset])
                    # ax.set_yticklabels(TreeNode.df_dict[dataset]['tid'], size = 12)
                ax.set_yticklabels(ax.get_yticklabels(), size = 8)
                plt.show()

        def dropdown_eventhandler(change):
            update_dashboard(change.new, row_num.value, dropdown_label.value)

        def row_num_eventhandler(change):
            update_dashboard(dropdown_df.value, change.new, dropdown_label.value)

        def dropdown_label_eventhandler(change):
            update_dashboard(dropdown_df.value, row_num.value, change.new)

        dropdown_df.observe(dropdown_eventhandler, names='value')
        row_num.observe(row_num_eventhandler, names='value')
        dropdown_label.observe(dropdown_label_eventhandler, names='value')

        tab.set_title(0, 'Tree View')
        tab.set_title(1, 'Trajectory View')
        tab.set_title(2, 'Trajectory Similarity')
        tab.set_title(3, 'EDA')
        tab.set_title(4, 'Dataset Exploration')
        tab.set_title(5, 'Sankey View')
        tab.set_title(6, 'Heatmap Plot')

        display_dashboard = widgets.VBox([input_widgets, tab])
        display(display_dashboard)


    def show(self, onlyLeaf=False):
        """
          Shows info about each cluster node.

          Parameters
          ----------
          onlyLeaf : bool
            A boolean attribute to determine if all nodes should be displayed
            or only leaf nodes.
        """

        if not onlyLeaf or len(self.division) <= 0:
            print('==' * self.depth, 'Node id: ', self.id)
            if self.parent is None:
                print('==' * self.depth, 'Parent id: None')
            else:
                print('==' * self.depth, 'Parent id: ', self.parent.id)
                print('==' * self.depth, 'Node depth: ', self.depth)
                print('==' * self.depth, 'Dataset shape (points x aspects): ', self.data.shape)
                print('==' * self.depth, 'Number of trajectories: ', self.trajList.size)
                print('==' * self.depth, '# of Users on this node =', self.data.label.unique().size)
                print('==' * self.depth, 'Users on this node =', self.data.label.unique())
            if len(self.division) > 0:
                print('==' * self.depth, 'Division done using the aspect', self.division['aspect'], ' with value ',
                      self.division['value'], '; Frequency threshold = ', self.threshold)
                print('==' * self.depth, 'Trajectories on the left node = ', self.left.trajList.size)
                print('==' * self.depth, 'Trajectories on the right node = ', self.right.trajList.size)
            else:
                print('==' * self.depth, 'Leaf node')
            print(' ')

        if self.left is not None:
            self.left.show(onlyLeaf)
        if self.right is not None:
            self.right.show(onlyLeaf)

    def graphicTree(self, graphTree):
        """
          Shows info about each cluster node in the tree generated by Digraph
          plot.

          Parameters
          ----------
          graphTree : Digraph
            A base class for directed graphs.
        """

        threshold = '{:.4f}'.format(self.thresholdVal)
        if self.left.thresholdVal is not None and self.right.thresholdVal is not None:
            thresholdLeft = '{:.4f}'.format(self.left.thresholdVal)
            thresholdRight = '{:.4f}'.format(self.right.thresholdVal)
        elif self.left.thresholdVal is not None and self.right.thresholdVal is None:
            thresholdLeft = '{:.4f}'.format(self.left.thresholdVal)
        elif self.left.thresholdVal is None and self.right.thresholdVal is not None:
            thresholdRight = '{:.4f}'.format(self.right.thresholdVal)

        tres_ = "\nMean_tres["
        if self.split == 'binary':
            left = 'NO'
            right = 'YES'
            thres = ''
        else:
            left = 'less than the average'
            right = 'greater than the average'
            thres = tres_ + str(threshold) + "]"

        if self.left.done == 'No' and self.right.done == 'No':
            graphTree.edge(str(self.id) + "\n" + self.parentName + tres_ + str(threshold) + "]",
                           str(self.left.id) + "\n" + self.leftChildName + tres_ + str(thresholdLeft) + "]",
                           left)
            graphTree.edge(str(self.id) + "\n" + self.parentName + tres_ + str(threshold) + "]",
                           str(self.right.id) + "\n" + self.rightChildName + tres_ + str(thresholdRight) + "]",
                           right)

        elif self.left.done == 'No' and self.right.done == 'Yes':
            graphTree.edge(str(self.id) + "\n" + self.parentName + tres_ + str(threshold) + "]",
                           str(self.left.id) + "\n" + self.leftChildName + tres_ + str(thresholdLeft) + "]",
                           left)
            graphTree.edge(str(self.id) + "\n" + self.parentName + tres_ + str(threshold) + "]",
                           # str(self.right.id)+"\n"+self.rightChildName+"\n["+self.division['value']+"]",right)
                           str(self.right.id) + "\n" + self.rightChildName, right)

        elif self.left.done == 'Yes' and self.right.done == 'No':
            graphTree.edge(str(self.id) + "\n" + self.parentName + tres_ + str(threshold) + "]",
                           # str(self.left.id)+"\n"+self.leftChildName+"\n["+self.division['value']+"]",left)
                           str(self.left.id) + "\n" + self.leftChildName, left)
            graphTree.edge(str(self.id) + "\n" + self.parentName + tres_ + str(threshold) + "]",
                           str(self.right.id) + "\n" + self.rightChildName + tres_ + str(thresholdRight) + "]",
                           right)

        else:
            graphTree.edge(str(self.id) + "\n" + self.parentName + tres_ + str(threshold) + "]",
                           # str(self.left.id)+"\n"+self.leftChildName+"\n["+self.division['value']+"]",left)
                           str(self.left.id) + "\n" + self.leftChildName + "\nU.T.", left)
            graphTree.edge(str(self.id) + "\n" + self.parentName + tres_ + str(threshold) + "]",
                           # str(self.right.id)+"\n"+self.rightChildName+"\n["+self.division['value']+"]",right)
                           str(self.right.id) + "\n" + self.rightChildName + "\nU.T.", right)

        if self.left.done == 'No':
            self.left.graphicTree(graphTree)
        if self.right.done == 'No':
            self.right.graphicTree(graphTree)

    def dashTree(self, strat_split, min_trajs, max_depth):
        """
          Method that calculates the frequency matrix and the clusters generated
          from the division of data based on a split criteria defined in the
          Tree constructor.

        """

        """
          columnns: gets only the relevant aspects (attributes)
          dummies: converts categorical variable into dummy/indicator variables.
          vals: removes the 'tid' column
          freqMatrix: creates a spreadsheet-style pivot table as a DataFrame
                      from dummies and vals.
        """
        # columns = self.data.drop(['time', 'poi', 'type', 'rating', 'label'], axis=1)
        # dummies = pd.get_dummies(columns, prefix_sep='~')
        # vals = dummies.drop(['tid'], axis=1)
        # self.freqMatrix = pd.pivot_table(dummies, index=['tid'], values=vals.columns, aggfunc=np.sum)
        self.generate_freq_matrix()
        self.split = strat_split
        self.maxTrajPerGroup = min_trajs
        self.maxDepth = max_depth

        if TreeNode.relative and TreeNode.absolute_frequency_matrix is None:
            TreeNode.absolute_frequency_matrix = self.freqMatrix.sum()
            dt = [res for res in self.freqMatrix.mean()]
            TreeNode.absolute_frequency_matrix = pd.DataFrame(columns=['mean'], data=dt, index=self.freqMatrix.columns)

        ### Stop criterion ###
        if (self.trajList.size <= 1 or
           (self.maxTrajPerGroup > 0 and self.trajList.size < self.maxTrajPerGroup) or
                (self.depth == self.maxDepth)
                ):

            self.parentName = str(self.data.tid.unique().size)
            self.done = 'Yes'
            self.check_label('c', self.depth)
            TreeNode.df_dict[str(self.id)] = self.freqMatrix
            TreeNode.nodeLabel[self.depth].append(self.id)

            idx = TreeNode.df_dict[str(self.id)].index
            df_filter = df[df.tid.isin(idx)]
            TreeNode.df_leaves[str(self.id)] = df_filter

            TreeNode.temporario += get_entropy(df_filter) * len(df_filter.tid.unique())
            TreeNode.clusters += 1

            TreeNode.dendrogram_dict[str(self.id)] = []

            return 0

        TreeNode.nodeNum += 1
        TreeNode.source.append(TreeNode.nodeNum)

        minVar = -1

        if TreeNode.relative:
            dt = [res for res in self.freqMatrix.sum() / len(df)]
            self.threshold = pd.DataFrame(columns=['mean'], data=dt, index=self.freqMatrix.columns)
        else:
            dt = [res for res in self.freqMatrix.mean()]
            self.threshold = pd.DataFrame(columns=['mean'], data=dt, index=self.freqMatrix.columns)

        self.variance = {}
        left_dict = {}
        right_dict = {}
        feature_list, initial_var, var_red = [], [], []
        reducao = {}
        split_value = {}
        muitas_result = {}
        msm_result = {}

        for col in self.freqMatrix.columns:

            if col in self.skipVal:
                continue

            left, left_idx, right, right_idx = [], [], [], []

            for i, reg in enumerate(self.freqMatrix[col]):
                if reg < self.threshold['mean'][col]:
                    left.append(reg)
                    left_idx.append(i)
                else:
                    right.append(reg)
                    right_idx.append(i)

            self.left_group[col] = left
            self.right_group[col] = right

            left_dict[col] = left_idx
            right_dict[col] = right_idx

            self.variance[col] = {
                "initial": self.freqMatrix[col].var(),
                "left": np.var(left),
                "right": np.var(right)
            }

            if self.split == 'var_red':

                save_df = self.freqMatrix.copy()
                save_df.reset_index(drop=True, inplace=True)
                esquerda = save_df.loc[save_df.index.isin(left_dict[col])]
                direita = save_df.loc[save_df.index.isin(right_dict[col])]

                for c in self.freqMatrix.columns:
                    initial_variance = self.freqMatrix[c].var()
                    variance_reduction = initial_variance - abs((np.var(esquerda[c]) - np.var(direita[c])) / 2)
                    reducao[c] = variance_reduction
                split_value[col] = sum(reducao.values()) / len(self.freqMatrix.columns)

            elif self.split == 'muitas':

                traj_left = [t for i, t in enumerate(self.freqMatrix.index.values) if i in left_dict[col]]
                esquerda = self.data.loc[self.data['tid'].isin(traj_left)]
                traj_right = [t for i, t in enumerate(self.freqMatrix.index.values) if i in right_dict[col]]
                direita = self.data.loc[self.data['tid'].isin(traj_right)]

                similarity_mean_node_muitas_esquerda = get_similarity_matrix(esquerda, 'MUITAS')
                try:
                    similarity_mean_node_muitas_esquerda = sum(similarity_mean_node_muitas_esquerda.mean()) / len(
                        similarity_mean_node_muitas_esquerda)
                except Exception as ex:
                    similarity_mean_node_muitas_esquerda = sum(similarity_mean_node_muitas_esquerda.mean()) / 1

                similarity_mean_node_muitas_direita = get_similarity_matrix(direita, 'MUITAS')
                similarity_mean_node_muitas_direita = sum(similarity_mean_node_muitas_direita.mean()) / len(
                    similarity_mean_node_muitas_direita)

                muitas_media = (similarity_mean_node_muitas_esquerda + similarity_mean_node_muitas_direita) / 2

                muitas_result[col] = muitas_media

            elif self.split == 'msm':

                traj_left = [t for i, t in enumerate(self.freqMatrix.index.values) if i in left_dict[col]]
                esquerda = self.data.loc[self.data['tid'].isin(traj_left)]
                traj_right = [t for i, t in enumerate(self.freqMatrix.index.values) if i in right_dict[col]]
                direita = self.data.loc[self.data['tid'].isin(traj_right)]

                similarity_mean_node_msm_esquerda = get_similarity_matrix(esquerda, 'MSM')
                try:
                    similarity_mean_node_msm_esquerda = sum(similarity_mean_node_msm_esquerda.mean()) / len(
                        similarity_mean_node_msm_esquerda)
                except Exception as ex:
                    similarity_mean_node_msm_esquerda = sum(similarity_mean_node_msm_esquerda.mean()) / 1
                similarity_mean_node_msm_direita = get_similarity_matrix(direita, 'MSM')
                similarity_mean_node_msm_direita = sum(similarity_mean_node_msm_direita.mean()) / len(
                    similarity_mean_node_msm_direita)

                msm_media = (similarity_mean_node_msm_esquerda + similarity_mean_node_msm_direita) / 2

                msm_result[col] = msm_media

            elif self.split == "binary":

                differenceBetweenGroups = np.abs(len(right) - len(left))
                if minVar == -1 or differenceBetweenGroups < minVar:
                    minVar = differenceBetweenGroups
                    self.division = col

            elif self.split == "minVariance":

                calcMinVar = (self.variance[col]["left"] + self.variance[col][
                    "right"]) / 2;  # average variance between groups
                if minVar == -1 or calcMinVar < minVar:
                    minVar = calcMinVar
                    self.division = col

            else:  # self.split == 'max_red'
                calcMinVar = self.variance[col]['initial'] - (self.variance[col]["left"] + self.variance[col][
                    "right"]) / 2  # average variance between groups

                if calcMinVar > minVar:
                    minVar = calcMinVar
                    self.division = col

        if self.split == 'muitas':
            self.division = max(muitas_result, key=muitas_result.get)
        elif self.split == 'msm':
            self.division = max(msm_result, key=msm_result.get)
        elif self.split == 'var_red':
            self.division = max(split_value, key=split_value.get)

        asp, val = self.division.split('~')
        self.thresholdVal = self.threshold['mean'][self.division]
        self.check_label(f'{asp} {val}', self.depth)

        if self.division not in self.skipVal:
            self.skipVal.append(self.division)

        TreeNode.value[self.depth].extend([len(self.left_group[self.division]), len(self.right_group[self.division])])
        TreeNode.nodeLabel[self.depth].append(self.id)
        TreeNode.df_dict[str(self.id)] = self.freqMatrix

        self.parentName = asp + "\n[" + val + "]"

        traj_left = [t for i, t in enumerate(self.freqMatrix.index.values) if i in left_dict[self.division]]

        self.left = TreeNode(self.data.loc[self.data['tid'].isin(traj_left)], self)
        self.left.dashTree(strat_split, min_trajs, max_depth)
        self.leftChildName = self.left.parentName

        TreeNode.dendrogram_dict[str(self.id)].append(self.left.id)

        traj_right = [t for i, t in enumerate(self.freqMatrix.index.values) if i in right_dict[self.division]]

        self.right = TreeNode(self.data.loc[self.data['tid'].isin(traj_right)], self)
        self.right.dashTree(strat_split, min_trajs, max_depth)
        self.rightChildName = self.right.parentName

        TreeNode.dendrogram_dict[str(self.id)].append(self.right.id)# TreeNode.dashboard(TreeNode.df_dict)

## Run tool

In [13]:
strat_split = 'var_red' # binary, minVariance, max_red, var_red
min_trajs = 70
max_depth = 4
TreeNode.dashboard(TreeNode.df_dict, strat_split, min_trajs, max_depth)